In [1]:
#libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import to_categorical
import gc


Using TensorFlow backend.


In [2]:
#more libraries so that it processes faster
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split as tts, cross_val_score as cvs
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score

In [3]:
#clean up sessions to improve performance

def reset_keras():
  sess = get_session()
  clear_session()
  sess.close()
  sess = get_session()

def store_CPU(x):
  with tf.device('/CPU:0'):
      x=x
      reset_keras()
      gc.collect()

# Pull in data and convert it for the purposes of of the model

In [4]:
#load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
#define input and output variables from training data

random_seed = 1234

X = train.drop('label', axis = 1).astype('float32')
y = train['label']

In [5]:
#transforma the data
#normalize X - pixels can only be between 0 and 255

X = X/255

#define labels and then one-hot encode
y = to_categorical(y, num_classes = 10)

In [11]:
#get total inputs and classes

n_input = X.shape[1]
n_classes = y.shape[1]

# Run model using training/test splits

In [12]:
#Define Model, 20 nodes and 1 hidden layer
model = Sequential()
model.add(Dense(20, input_dim=n_input, activation='relu', kernel_initializer='normal'))
model.add(Dense(20, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(n_classes, activation = 'softmax', kernel_initializer = 'normal'))

sgd = SGD(lr = 0.01)
model.compile(loss = 'categorical_crossentropy',
             optimizer = sgd,
             metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [14]:
#fit model
start = timer()
Xtrain, Xtest, ytrain, ytest = tts(X, y, test_size = .2, random_state = random_seed)
train = model.fit(Xtrain, ytrain, validation_data = (Xtest, ytest), epochs = 20, batch_size = 200, verbose = 1)
end = timer()


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 7s 198us/step - loss: 2.2736 - acc: 0.2019 - val_loss: 2.2306 - val_acc: 0.2900
Epoch 2/20
33600/33600 [==============================] - 5s 141us/step - loss: 2.1281 - acc: 0.3769 - val_loss: 1.9781 - val_acc: 0.4395
Epoch 3/20
33600/33600 [==============================] - 7s 211us/step - loss: 1.7202 - acc: 0.5276 - val_loss: 1.4358 - val_acc: 0.6511
Epoch 4/20
33600/33600 [==============================] - 5s 159us/step - loss: 1.2043 - acc: 0.6987 - val_loss: 1.0132 - val_acc: 0.7365
Epoch 5/20
33600/33600 [==============================] - 6s 170us/step - loss: 0.8881 - acc: 0.7605 - val_loss: 0.7838 - val_acc: 0.7837
Epoch 6/20
33600/33600 [==============================] - 7s 214us/step - loss: 0.7154 - acc: 0.8004 - val_loss: 0.6591 - val_acc: 0.8135
Epoch 7/20
33600/33600 [==============================] - 7s 202us/step - loss: 0.6176 - acc: 0.8236 - val_loss: 0.5884 - v

In [15]:
#test model scores
train_score = model.evaluate(Xtrain, ytrain, verbose = 0)
test_score = model.evaluate(Xtest, ytest, verbose = 0)
print('Time to train: {:.3f}'.format(end-start))
print('Training Scores: Accuracy = {:.3f}, Loss = {:.3f}'.format(train_score[1], train_score[0]))
print('Test Scores: Accuracy = {:.3f}, Loss = {:.3f}'.format(test_score[1], test_score[0]))

Time to train: 136.468
Training Scores: Accuracy = 0.903, Loss = 0.336
Test Scores: Accuracy = 0.895, Loss = 0.361


# Create a class that handles all of the work for each version of the model

In [34]:
#build a class that will produce all models

class Network:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def model(self, layers, nodes, optimizer, lr):
        model = Sequential()
        model.add(Dense(nodes, input_dim=n_input, activation='relu', kernel_initializer='normal'))
        for _ in range(1,layers+1):
            model.add(Dense(nodes, activation='relu', kernel_initializer='normal'))
        model.add(Dense(n_classes, activation = 'softmax', kernel_initializer = 'normal'))
        optimizer = optimizer(lr = lr)
        model.compile(loss = 'categorical_crossentropy',
                     optimizer = optimizer,
                     metrics=['accuracy'])
        print('Parameters: Layers = {:2}, Nodes = {:2}'.format(layers, nodes))
        return model
    
    def fit (self, model, x, y):
        start = timer()
        train = model.fit(x, y, epochs = 20, batch_size = 200, verbose = 0)
        end = timer()
        train_score = model.evaluate(Xtrain, ytrain, verbose = 0)
        print('Time to train: {:.3f}'.format(end-start))
        print('Training Scores: Accuracy = {:.3f}, Loss = {:.3f}'.format(train_score[1], train_score[0]))

    def kaggle (self, model, filename, X):
        pred = model.predict_classes(X)
        imageid = [int(i) for i in range(1,len(pred)+1)]
        file = pd.DataFrame({'ImageID': imageid,
                            'Label': pred})                        
        file.to_csv(filename, index = False, header = True)  

# Model #1

In [35]:
#define class
run_one = Network(X,y)

In [36]:
#set model parameters
run_one.model(2, 20, SGD, 0.01)

Parameters: Layers =  2, Nodes = 20


In [37]:
#fit model
run_one.fit(model, X, y)

Time to train: 122.703
Training Scores: Accuracy = 0.959, Loss = 0.143


In [38]:
#produce output
run_one.kaggle(model, 'NN_Submission_1.csv', test)

# Model #2

In [39]:
#define class
run_two = Network(X,y)

In [40]:
#set model parameters
run_two.model(5,20, SGD, 0.01)

Parameters: Layers =  5, Nodes = 20


In [41]:
#fit model
run_two.fit(model, X, y)

Time to train: 80.939
Training Scores: Accuracy = 0.963, Loss = 0.131


In [42]:
#produce output
run_two.kaggle(model, 'NN_Submission_2.csv', test)

# Model #3

In [43]:
#define class
run_three = Network(X,y)

In [44]:
#set model parameters
run_three.model(2, 40, SGD, 0.01)

Parameters: Layers =  2, Nodes = 40


In [45]:
#fit model
run_three.fit(model, X, y)

Time to train: 136.269
Training Scores: Accuracy = 0.965, Loss = 0.121


In [46]:
#produce output
run_three.kaggle(model, 'NN_Submission_3.csv', test)

# Model #4

In [47]:
#define class
run_four = Network(X,y)

In [48]:
#set model parameters
run_four.model(5, 40, SGD, 0.01)

Parameters: Layers =  5, Nodes = 40


In [49]:
#fit model
run_four.fit(model, X, y)

Time to train: 139.961
Training Scores: Accuracy = 0.968, Loss = 0.112


In [50]:
#produce output
run_four.kaggle(model, 'NN_Submission_4.csv', test)